# Altair: Your Paper's Best Friend
There are a number of modules and packages that help you create charts and graphs and other visualizations in Python. Some popular ones include:
  * Matplotlib
  * Seaborn (sits on top of Matplotlib)
  * Bokeh
  * Altair
  
I have used a number of different approaches, and I've found Altair to be the most intuitive, quick to use, and aesthetically pleasing. So that's what we'll introduce today.

In [ ]:
import pandas as pd
import altair as alt
import numpy as np

<img src="excellence.jpg">

## Five Principles
Within reason:

1. Above all else show the data
2. Maximize your data-ink ratio
3. Erase non-data ink
4. Erase redundant data ink
5. Revise and edit

## Visualizing VIA Reliability on the Kitchener Corridor
Using the outputs we created in the Pandas tutorial, we are going to produce some visualizations and summaries that help us communicate reliability on the Kitchener Corridor portion of the VIA Rail network. Let's load our data now






In [ ]:
otp = pd.read_csv('kitchener_otp.csv')
times = pd.read_csv('kitchener_times.csv')

## Charts, Marks, and Encodings
First, let's start with an introduction to Altair's `Chart` object. The `Chart` class holds the basis for creating a chart, and takes a Pandas dataframe as a data source. From there we can make marks, and encode data into channels.

In [ ]:
alt.Chart(otp).mark_bar().encode(
    alt.X('otp:Q'),
    alt.Y('station:N'),
    alt.Row('train:N'),
    alt.Color('station:N')
)

## Example: Station Reliability by Train
Let's find an interesting and useful way to show station reliability by train using a much nicer version of a "box and whiskers" plot:

<img src="box_and_whisker.jpg">

Our steps will be as follows:
1. Construct a `DataFrame` with the values we need
2. Plot each element of the plot (bars and dots) individually
3. Combine them together into a plot
4. Erase what we don't need